In [8]:
import numpy as np
import scipy
import os

cwd = os.getcwd()

In [9]:
def QPSK(input): # Map an narray of complex numbers to corresponding binary bits pairs
    imag = input.imag
    digit1 = imag < 0
    digit1 = digit1.astype(int)
    real = input.real
    digit2 = real < 0
    digit2 = digit2.astype(int)
    return np.array([str(digit1[i]) + str(digit2[i]) for i in range(len(digit1))])


In [10]:
# Load the data
file_number = 4

data_dir = os.path.join(cwd, 'data')
file_name = "file{}.csv".format(file_number)
input_signal = np.loadtxt(open(os.path.join(data_dir, file_name), "rb"), delimiter=",")
channel = np.loadtxt(open(os.path.join(data_dir, "channel.csv"), "rb"), delimiter=",")

channel_len = channel.size
block_len = 1024
prefix_len = 32
total_len = block_len + prefix_len

In [11]:
channel_time = np.zeros(block_len)
channel_time[:channel_len] = channel
channel_fft = np.fft.fft(channel_time)

data_length = input_signal.size

result = np.array([])

In [12]:
for i in range(data_length // total_len):
    block_before_discard = input_signal[total_len * i : total_len * (i + 1)]
    block = block_before_discard[prefix_len:]
    fft_result = np.fft.fft(block)
    output = np.divide(fft_result, channel_fft)

    info_block = output[1:512]

    result = np.append(result, info_block)

mapped = QPSK(result)

In [13]:
output_bitstring = "".join(mapped)
output_dir = os.path.join(cwd, 'week1_output')
raw_file_name = "raw_output_{}.txt".format(file_number)

output = np.savetxt(os.path.join(output_dir, raw_file_name),np.array([output_bitstring]),fmt="%s")

In [14]:
# Process the bitstring raw output
import utils.binary_decode
cropped = utils.binary_decode.crop_header(output_bitstring)
bin_file_name = "result_{}.bin".format(file_number)
utils.binary_decode.to_binary(cropped, os.path.join(output_dir, bin_file_name))


The header is 
files/4738210983.tiff 130932 
